# **GRU based BaseLine Model - NIID setting**

## Download and import the necessary libraries

In [ ]:
!pip install transformers

In [ ]:
import torch
from transformers import BertTokenizer, BertModel
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states = True,)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import scipy as sp
import pandas as pd

from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, roc_auc_score, average_precision_score, precision_recall_curve
import copy

import torch

import torch.nn as nn
import torch.nn.functional as F
import random
from operator import add

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings("ignore")

## Data loading and basic processing

In [ ]:
# This path contains the CSV file
PATH1 = '/content/drive/MyDrive/Tweets/sent_final_preds.csv'
PATH2 = '/content/drive/MyDrive/Tweets/cd.csv'

df = pd.read_csv(PATH1, sep = ",")
dff = pd.read_csv(PATH2, sep = ",")


# copying the missing column 'domain' from dff to df
d = list(dff['domain'].unique())
d.remove('random_tweet')
d.remove('random_reply')

copied = dff['domain'].values
sum = 0
for c in range(len(copied)):
    if copied[c] == 'random_tweet' or copied[c] == 'random_reply':
        r = random.sample(range(0, len(d)), 1)
        copied[c] = d[r[0]]
        sum += 1

df['domain'] = copied

df.head(2)

In [ ]:
df_list = []

for uniq in df['domain'].unique():
    df_list.append(df.loc[df['domain'] == uniq])

In [ ]:
labels = []

# sentim - list of lists of sentiments (0/1/2)

for dummy_df in df_list:
    dummy_label = []
    
    for i in range(len(dummy_df)):    
        dummy_label.append(dummy_df.Labels.values[i])

    labels.append(dummy_label)


In [ ]:
for i, label in enumerate(labels):
    print('Client', i, ', #zeros:', len(label) - np.sum(np.array(label)), ', #ones:', np.sum(np.array(label)))

In [ ]:
# Funcction which takes sentence as input and gives word wise BERT embedding by summing up the last 4 hidden states

def bert_embedded(text, model):

    marked_text = "[CLS] " + text + " [SEP]"
    tokenized_text = tokenizer.tokenize(marked_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    
    model.eval()
    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        hidden_states = outputs[2]

    token_embeddings = torch.stack(hidden_states, dim=0)
    token_embeddings = torch.squeeze(token_embeddings, dim=1)
    token_embeddings = token_embeddings.permute(1,0,2)

    token_vecs_sum = []

    for token in token_embeddings:
        sum_vec = torch.sum(token[-4:], dim=0)
        token_vecs_sum.append(sum_vec)

    return token_vecs_sum

In [ ]:
embedded_dfs = []

for dummy_df in df_list:
    dummy_tweets = []
    for tweet in dummy_df['Tweets']:
        dummy_tweets.append(bert_embedded(tweet, model))
    embedded_dfs.append(dummy_tweets)
    

In [ ]:
len(embedded_dfs[0])

In [ ]:
all_vect = []

for df in embedded_dfs:

    client_vect = []

    for tweet in df: 

        vect = np.stack(tweet)
        
        if len(vect) < 40:     
            new_vect = np.concatenate((vect, np.zeros((40 - len(vect), vect.shape[1]))))
        
        else:
            new_vect = vect[:40]

        client_vect.append(new_vect)
    
    all_vect.append(client_vect)

In [ ]:
cl1_x = np.stack(all_vect[0])
cl1_y = np.stack(labels[0])

cl2_x = np.stack(all_vect[1])
cl2_y = np.stack(labels[1])

cl3_x = np.stack(all_vect[2])
cl3_y = np.stack(labels[2])

cl4_x = np.stack(all_vect[3])
cl4_y = np.stack(labels[3])

cl5_x = np.stack(all_vect[4])
cl5_y = np.stack(labels[4])

cl6_x = np.stack(all_vect[5])
cl6_y = np.stack(labels[5])

cl7_x = np.stack(all_vect[6])
cl7_y = np.stack(labels[6])

cl8_x = np.stack(all_vect[7])
cl8_y = np.stack(labels[7])

cl9_x = np.stack(all_vect[8])
cl9_y = np.stack(labels[8])


In [ ]:
# 2 layered GRU with hidden size = 256
# and dropout of 0.2

class GRUNet(nn.Module):

    def __init__(self, input_dim):

        super(GRUNet, self).__init__()

        self.gru = nn.GRU(input_dim, 128, 2, batch_first=True, dropout=0.2)
        self.fc = nn.Linear(128, 2)
        self.relu = nn.ReLU()
        

    def forward(self, x, h):

        out, h = self.gru(x, h)
        out = self.fc(self.relu(out[:,-1]))

        return out, h
    

    def init_hidden(self, b_size = 128):

        weight = next(self.parameters()).data
        hidden = weight.new(2, b_size, 128).zero_()

        return hidden


In [ ]:
batch_size = 128

cl1_tx, cl1_vx, cl1_ty, cl1_vy = train_test_split(cl1_x, cl1_y, test_size = 0.2, random_state = ran_seed)
cl1_train_data = TensorDataset(torch.from_numpy(cl1_tx), torch.from_numpy(cl1_ty))
cl1_data_loader = DataLoader(cl1_train_data, shuffle=True, batch_size = batch_size, drop_last=True)

cl2_tx, cl2_vx, cl2_ty, cl2_vy = train_test_split(cl2_x, cl2_y, test_size = 0.2, random_state = ran_seed)
cl2_train_data = TensorDataset(torch.from_numpy(cl2_tx), torch.from_numpy(cl2_ty))
cl2_data_loader = DataLoader(cl2_train_data, shuffle=True, batch_size = batch_size, drop_last=True)

cl3_tx, cl3_vx, cl3_ty, cl3_vy = train_test_split(cl3_x, cl3_y, test_size = 0.2, random_state = ran_seed)
cl3_train_data = TensorDataset(torch.from_numpy(cl3_tx), torch.from_numpy(cl3_ty))
cl3_data_loader = DataLoader(cl3_train_data, shuffle=True, batch_size = batch_size, drop_last=True)

cl4_tx, cl4_vx, cl4_ty, cl4_vy = train_test_split(cl4_x, cl4_y, test_size = 0.2, random_state = ran_seed)
cl4_train_data = TensorDataset(torch.from_numpy(cl4_tx), torch.from_numpy(cl4_ty))
cl4_data_loader = DataLoader(cl4_train_data, shuffle=True, batch_size = batch_size, drop_last=True)

cl5_tx, cl5_vx, cl5_ty, cl5_vy = train_test_split(cl5_x, cl5_y, test_size = 0.2, random_state = ran_seed)
cl5_train_data = TensorDataset(torch.from_numpy(cl5_tx), torch.from_numpy(cl5_ty))
cl5_data_loader = DataLoader(cl5_train_data, shuffle=True, batch_size = batch_size, drop_last=True)

cl6_tx, cl6_vx, cl6_ty, cl6_vy = train_test_split(cl6_x, cl6_y, test_size = 0.2, random_state = ran_seed)
cl6_train_data = TensorDataset(torch.from_numpy(cl6_tx), torch.from_numpy(cl6_ty))
cl6_data_loader = DataLoader(cl6_train_data, shuffle=True, batch_size = batch_size, drop_last=True)

cl7_tx, cl7_vx, cl7_ty, cl7_vy = train_test_split(cl7_x, cl7_y, test_size = 0.2, random_state = ran_seed)
cl7_train_data = TensorDataset(torch.from_numpy(cl7_tx), torch.from_numpy(cl7_ty))
cl7_data_loader = DataLoader(cl7_train_data, shuffle=True, batch_size = batch_size, drop_last=True)

cl8_tx, cl8_vx, cl8_ty, cl8_vy = train_test_split(cl8_x, cl8_y, test_size = 0.2, random_state = ran_seed)
cl8_train_data = TensorDataset(torch.from_numpy(cl8_tx), torch.from_numpy(cl8_ty))
cl8_data_loader = DataLoader(cl8_train_data, shuffle=True, batch_size = batch_size, drop_last=True)

cl9_tx, cl9_vx, cl9_ty, cl9_vy = train_test_split(cl9_x, cl9_y, test_size = 0.2, random_state = ran_seed)
cl9_train_data = TensorDataset(torch.from_numpy(cl9_tx), torch.from_numpy(cl9_ty))
cl9_data_loader = DataLoader(cl9_train_data, shuffle=True, batch_size = batch_size, drop_last=True)

In [ ]:
valid_xsets = [cl1_vx, cl2_vx, cl3_vx, cl4_vx, cl5_vx, cl6_vx, cl7_vx, cl8_vx, cl9_vx]
valid_ysets = [cl1_vy, cl2_vy, cl3_vy, cl4_vy, cl5_vy, cl6_vy, cl7_vy, cl8_vy, cl9_vy]

In [ ]:
data_loader = [cl1_data_loader, cl2_data_loader, cl3_data_loader, cl4_data_loader, cl5_data_loader
               , cl6_data_loader, cl7_data_loader, cl8_data_loader, cl9_data_loader]

In [ ]:
cl1_model = GRUNet(next(iter(data_loader[0]))[0].shape[2])
cl2_model = GRUNet(next(iter(data_loader[1]))[0].shape[2])
cl3_model = GRUNet(next(iter(data_loader[2]))[0].shape[2])
cl4_model = GRUNet(next(iter(data_loader[3]))[0].shape[2])
cl5_model = GRUNet(next(iter(data_loader[4]))[0].shape[2])
cl6_model = GRUNet(next(iter(data_loader[5]))[0].shape[2])
cl7_model = GRUNet(next(iter(data_loader[6]))[0].shape[2])
cl8_model = GRUNet(next(iter(data_loader[7]))[0].shape[2])
cl9_model = GRUNet(next(iter(data_loader[8]))[0].shape[2])

In [ ]:
all_models = [cl1_model, cl2_model, cl3_model, cl4_model, cl5_model, cl6_model, cl7_model, cl8_model, cl9_model]

In [ ]:
def evaluate(model, testx, testy):

    model.eval()
    preds = []

    for i in range(len(testx)):

        inp = torch.from_numpy(np.array(testx[i]))
        h = model.init_hidden(b_size = 1)

        inp = inp.reshape((1, inp.shape[0], inp.shape[1]))

        out, h = model(inp.float(), h)

        preds.append(torch.argmax(out))
    


    report = classification_report(testy, torch.tensor(preds), labels=[0, 1])

    f_0 = int(report.split('.')[3].split(' ')[0])/100
    f_1 = int(report.split('.')[6].split(' ')[0])/100

    return f_0, f_1

In [ ]:
def FEDAVG(train_loader, model):
    
  
    optimizer = torch.optim.Adam(model.parameters(), lr=10e-4)
    
    model.train()

    epoch_times = []

    for epoch in range(10):
        
        t_loss = 0

        h = model.init_hidden()

        for x, label in train_loader:

            h = h.data
            model.zero_grad()
            
            out, h = model(x.float(), h)

            model.zero_grad()
            
            loss = torch.nn.functional.cross_entropy(out, label)

            loss.backward()
            optimizer.step()
            
            t_loss += loss.detach().numpy() 

## FEDAVG

In [ ]:
# FEDAVG

zero_client = GRUNet(next(iter(data_loader[0]))[0].shape[2])
fed_glob = GRUNet(next(iter(data_loader[0]))[0].shape[2])

for p in zero_client.parameters():
    p.data = p.data * 0

fedavg_f1_0 = 0
fedavg_f1_1 = 0
fedavg = 0

for comm_rounds in range(50):



    train_ind = random.sample(range(0, len(data_loader)), 7)
    train_ind = sorted(train_ind)

    test_ind = [i for i in range(len(data_loader))]
    
    for ind in train_ind:
        test_ind.remove(ind)
        
    train_clients = []
    test_clients = []

    for ind in train_ind:
        train_clients.append(data_loader[ind])
    
    for clients in data_loader:
        if clients not in train_clients:
            test_clients.append(clients)

    
    for ind in train_ind:
        
        dummy = []
        
        FEDAVG(data_loader[ind], all_models[ind])

    fed_glob.load_state_dict(zero_client.state_dict()) 

    for ind in train_ind:
        for p1, p2 in zip(fed_glob.parameters(), all_models[ind].parameters()):
            p1.data = p1.data + p2.data


    for p in fed_glob.parameters():
        p.data = p.data / len(train_ind)

    for client_model in all_models:
        client_model.load_state_dict(fed_glob.state_dict()) 


    
    for ind, client_model in zip(test_ind, test_clients):

      
        f10, f11 = evaluate(fed_glob, valid_xsets[ind], valid_ysets[ind])
        fedavg_f1_0 += f10
        fedavg_f1_1 += f11
        fedavg += 1


    print('Communication Round', comm_rounds,'-> Class0 avg F1 :', np.round(fedavg_f1_0/fedavg, 3), '; Class1 avg F1:', np.round(fedavg_f1_1/fedavg, 3))